# Coding LLM architecture

In [ ]:
import torch
import torch.nn as nn

from classes.attention import MultiHeadAttention
from classes.activation import GELU

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.token_emb = nn.Embedding(cfg['vocab_size'], cfg["emb_dim"])
        self.position_emb = nn.Embedding(cfg['context_length'], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )

        self.final_norm = LayerNormalization(cfg["emb_dim"])
        self.output_layer = nn.Linear(cfg['emb_dim'], cfg["vocab_size"], bias=False)
    
    def forward(self, in_indexes):
        bacth_size, sequence_shape = in_indexes.shape

        tok_embeds = self.token_emb(in_indexes)
        pos_embeds = self.position_emb(torch.arange(sequence_shape, device=in_indexes.device))

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)

        logits = self.output_layer(x)
        return logits

    def backward(self):
        pass
    

class LayerNormalization(nn.Module):
    def __init__(self, length):
        super().__init__()

        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(length))
        self.shift = nn.Parameter(torch.zeros(length))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)

        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift    

# Feed Forward (the basic building block for the transformer block)
class FeedForward(nn.Module):
    def __init__(self, cfg, expansion_rate=4):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], expansion_rate * cfg["emb_dim"]),
            GELU(),
            nn.Linear(expansion_rate * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.attention = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.fforward = FeedForward(cfg)
        self.norm_1 = LayerNormalization(cfg["emb_dim"])
        self.norm_2 = LayerNormalization(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        short = x
        x = self.norm_1(x)
        x = self.attention(x)
        x = self.drop_shortcut(x)
        x  = x + short

        short = x
        x = self.norm_2(x)
        x = self.fforward(x)
        x = self.drop_shortcut(x)
        x = x + short
        return x

In [27]:
GPT_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False 
}

In [28]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
torch.manual_seed(123)

model = GPTModel(cfg=GPT_CONFIG)
logits = model(batch)
print(logits.shape)

torch.Size([2, 4, 50257])


In [32]:
total_params = sum(p.numel() for p in model.parameters())
total_params

163009536

## Text Generation

In [42]:
def generate_text_simple(model, idx, max_new_tokens, context_size):

    for _ in range(max_new_tokens):
        context = idx[:, -context_size:]

        with torch.no_grad():
            logits = model(context)
        prob = torch.softmax(logits[:, -1, :], dim=-1)
        new_idx = torch.argmax(prob, dim=-1, keepdim=True)

        idx = torch.cat((idx, new_idx), dim=1)

    return idx


In [39]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
encoded_tensor

tensor([[15496,    11,   314,   716]])

In [43]:
new_encoded = generate_text_simple(model, encoded_tensor, 10, context_size=GPT_CONFIG["context_length"])
new_encoded 

tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 38891, 34320,
          6074, 16187, 32054,  3059]])

In [47]:
tokenizer.decode(new_encoded.squeeze(0).tolist())

'Hello, I am Featureiman Byeswick palpMust payment shadowszman Sch'